# Diabetes Challenge

Your task today is to **analyze** the Kaggle "Pima Indians Diabetes Database" and to **predict** whether a patient has Diabetes or not.

## Task:
- Load the data from the database. The schema is called `diabetes`. To connect to the database you need to copy the `.env` file from the visualization or hands-on-ml repository into this repo. Explore the database, try to establish what the relationships between the tables are (1-1, 1-N, N-M). Explain to yourself and the group what data do you see and whether it makes sense. What JOINs are appropriate to use and why? 
- Use at least two different classification algorithms we have learned so far to predict Diabetes patients. 
- Discuss before you start with the modeling process which **evaluation metric** you choose and explain why.
- Implement a GridSearchCV or RandomizedSearchCV to tune the hyperparameters of your model.
- **Optional:** If you have time at the end, try to use sklearn's pipline module to encapsulate all the steps into a pipeline.

Don't forget to split your data in train and test set. And analyze your final model on the test data. It might also be necessary to scale your data in order to improve the performance of some of the models.


## Helpful links and advise:
- [sklearn documentation on hyperparameter tuning](https://scikit-learn.org/stable/modules/grid_search.html#grid-search)
- It might be helpful to check some sources on how to deal with imbalanced data. 
    * [8 Tactics to Combat Imbalanced Classes](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)
    * [Random-Oversampling/Undersampling](https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/)


# Data Description

## Context
This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

## Acknowledgements
Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

## About this dataset
The datasets consist of several medical predictor (independent) variables and one target (dependent) variable, Outcome. Independent variables include the number of pregnancies the patient has had, their BMI, insulin level, age, and so on. For the outcome class value 1 is interpreted as "tested positive for diabetes".

|Column Name| Description|
|:------------|:------------|
|Pregnancies|Number of times pregnant|
|Glucose|Plasma glucose concentration a 2 hours in an oral glucose tolerance test|
|BloodPressure|Diastolic blood pressure (mm Hg)|
|SkinThickness|Triceps skin fold thickness (mm)|
|Insulin|2-Hour serum insulin (mu U/ml)|
|BMI|Body mass index (weight in kg/(height in m)^2)|
|DiabetesPedigreeFunction| Diabetes pedigree function|
|Age| Age (years)|
|Outcome|Class variable (0 or 1) |

In [ ]:
# Import of relevant packages
import os
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from sqlalchemy import create_engine

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

from timeit import default_timer as timer

# Set random seed 
RSEED = 42
warnings.filterwarnings("ignore")

In [ ]:
# Import .env
# Inspect and make sql query in DBEAVER
# Load Data
# Train test split data
# Inspect data
# Pairplot
# Clean Data

In [ ]:
# Read database string from .env file (no need to change anything)
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

db = create_engine(DB_STRING)

# SQL-Statement
sql_str = """SET SCHEMA 'diabetes';
SELECT p.id AS patient_id, p."Age", p.pregnancies, p.bmi,
pdg.diabetespedigreefunction AS pdg_function, pdg.outcome AS outcome,
sk.skinthickness AS skinthickness,
bm.insulin AS insulin,
bm.glucose AS glucose,
bm.bloodpressure AS bpres
FROM patient AS p
INNER JOIN pedigree_outcome AS pdg
ON p.id = pdg.patientid
INNER JOIN skin AS sk
ON p.id = sk.patientid
INNER JOIN blood_metrics AS bm 
ON p.id = bm.patientid
AND bm.measurement_date = '2022-12-13'
"""

In [ ]:
# Read Table from Query
# Import with pandas
df_diab = pd.read_sql(sql_str, db)
df_diab.head()

df_diab.to_csv("diabetes_data.csv", index=False)

In [ ]:
### Inspect data
df_diab.head()
df_diab.info() ### All non null, but entries with 0 in it
df_diab.describe()


In [ ]:
### Remove 0 entries from dataset? - function in preprocessing

def detect_zero(col): # Return Index
    ind = col == 0
    return sum (ind)


In [ ]:
df_diab.apply(lambda x: detect_zero(x))

- Doesnt Make sense for bmi 11, skinthickness 227, glucose 5, bpres 35
- replace 0 with median

In [ ]:
### Pairplot without patient_id
sns.pairplot(df_diab.drop("patient_id", axis = 1), hue = "outcome")

In [ ]:
### Train test split
X = df_diab.drop(["patient_id", "outcome"], axis = 1)
y = df_diab.outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RSEED)

In [ ]:
### Remove zeros: "mi", "skinthickness", "glucose", "bpres"
col_zero = ["bmi", "skinthickness", "glucose", "bpres"]

### Colums to scale, all except pdg
col_scale = list (X_train.columns.drop("pdg_function"))

### Columns to categorize: pdg_function
col_cat = ["pdg_function"]

In [ ]:
### Calculate median column wise, for test_data only, return as a dict, leave out zeros
def col_median(col):
        ind = col > 0 ## Calculate Median from entries larger 0
        col_med = np.median (col[ind]) # Calculate median
        return col_med

In [ ]:
### scaling - standadization, applied in function. Differs if applied on training or test data set
def scale_func (X_df, scaler = None, train = True):
    
    #X_scaled = X_df[col_scale]
    if train == True: ### Scaling the training or test data?
        scaler_1 = StandardScaler() # If scaling the training data, first the Scaler-Object is opened
        X_scaled = scaler_1.fit_transform(X_df)

        return scaler_1, X_scaled # Return ScalerObject and Scaled X
    else:
        X_scaled = scaler.transform(X_df)
        return X_scaled

In [ ]:
### One hot encoding with enc = OneHotEncoder - similar structure as above , fit_transform for training data and transform for test
### scaling - standadization, applied in function. Differs if applied on training or test data set
def encode_func (X_df, col_cat, encoder = None, cat_dict = None, train = True):
    
    #X_encoded = X_df[col_cat]
    if train == True: ### Encode training data and save category levels
        # cast to categorical data and save the categories
        cat_dict = {} # Save categories in dictionary

        for col in col_cat:
            X_df[col] = X_df[col].astype("category")
            cat_dict[col] = X_df[col].cat.categories

        encoder_1 = OneHotEncoder(sparse = False) # If scaling the training data, first the Scaler-Object is opened
        X_encoded = encoder_1.fit_transform(X_df)

        return cat_dict, encoder_1, X_encoded # Return ScalerObject and Scaled X

    else:
        ### Transform to specified CategoricalDType()
        for col in col_cat:
            cat_type = CategoricalDtype(cat_dict[col], ordered = False)
            X_df[col] = X_df[col].astype(cat_type)

        # And then apply OneHotencoding
        X_encoded = encoder.transform(X_df)

        return X_encoded

In [ ]:
### Preprocessing
# in Function: 1. 
#               a) split off columns to replace zeros 
#               b) and categorical columns to dummy encode
#              2. Replace zeros in a)
#              3. Concat columns from 2. and remaining numeric columns
#              4. apply scaling on 3.)
#              5. apply dummy encoding on 1. b)
#              6. concat 4. and 5.

# Retain column names!

def preprocess_diab(X_df, col_zero, col_scale, col_cat, medians_train = None, scaler = None, encoder = None, cat_dict = None, train = True):
    # in Function: 1. 
    #               a) split off columns to replace zeros
    X_zero = X_df[col_zero] # Works
    # 1. b) and categorical columns to dummy encode
    X_cat = X_df[col_cat]

    # 2. Replace zeros in 1. a)
    # Calculate Median Column wise if using the train set
    if train == True:
        medians_train = X_zero.apply(lambda x: col_median(x))
    
    # Replace with values from median train - Stores in X_zero
    for col in col_zero:
        repl_val = medians_train[col]
        X_zero[col].replace(0, repl_val, inplace = True)
        del repl_val
    
    # 3. Concat columns from 2. X_zero and remaining numeric columns
    # Columns which do not have the zero colum and should be scaled
    col_scale_no_zero = [col for col in col_scale if col not in col_zero]
    # As their own df
    X_no_zero = X_df[col_scale_no_zero] 

    X_scale = pd.concat([X_no_zero, X_zero], axis = 1)
    ### Remember column names in the right order
    x_scale_names = col_scale_no_zero + col_zero
    
    # 4. apply scaling on 3.)
    if train == True:
        scaler, X_scale = scale_func(X_scale, train = train)
    
    else:
        X_scale = scale_func(X_scale, scaler, train = train)
    
    # 5. apply dummy encoding on 1. b)
    if train == True:
        cat_dict, encoder, X_encoded = encode_func(X_cat, col_cat, train = train)
    
    else:
        X_encoded = encode_func(X_cat, col_cat, encoder, cat_dict, train = train)
    
    #  Get dummy names for each encoded column - > list of new column names for encoded data
    x_encoded_names = []
    for key, val in cat_dict.items():
        for categ in val:
            col_nam = key + "_" + str(categ)
            x_encoded_names.append(col_nam)

    ### Concat X_scaled and X_cat
    X_preprocessed = np.concatenate([X_scale, X_encoded], axis = 1)
    ### Reapply Names to feature data - seems to be not that easy...
    preprocessed_names = x_scale_names + x_encoded_names

    if train == True:
        return X_preprocessed, preprocessed_names, cat_dict, encoder, scaler, medians_train
    else:
        return X_preprocessed


    


In [ ]:
X_train_preprocessed, preprocessed_names, cat_dict, encoder, scaler, medians_train = preprocess_diab(X_train, col_zero, col_scale, col_cat, train = True)

In [ ]:
X_test_preprocessed = preprocess_diab(X_test, col_zero, col_scale, col_cat, medians_train, scaler, encoder, cat_dict, train = False)

- Use at least two different classification methods
    - decision trees
    - K-nearest neighbors

- Choose and evaluation metric:
    - Recall to minimize the percentage of false negatives

In [ ]:
### HyperParameterTuning K-Nearest neighbors
param_grid_knn = {"n_neighbors" : [2,5,7,10,15,20],
                "weights" : ["uniform", "distance"],
                "p" : [1,2,3,4,5]
             }

# Instantiate gridsearch and define the metric to optimize 
gs_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, scoring='recall',
                  cv=5, verbose=5, n_jobs=-1)

# Fit gridsearch object to data.. also lets see how long it takes
start = timer()
gs_knn.fit(X_train_preprocessed, np.ravel(y_train))
end = timer()
gs_time = end-start


In [ ]:
# Best score
print('Best score:', round(gs_knn.best_score_, 3))

# Best parameters
print('Best parameters:', gs_knn.best_params_)

In [ ]:
# Save best KNN model and predict
gs_knn_best = gs_knn.best_estimator_

# Making predictions on the test set
y_pred_test = gs_knn_best.predict(X_test_preprocessed)

# Print accuracy score 
print("Accuracy:", accuracy_score(y_test, y_pred_test).round(2))
print("Recall:", recall_score(y_test, y_pred_test).round(2))
print("-----"*10)

# Print confusion matrix
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, cmap='YlGn');


In [123]:
### For logistic Regression - SGD Classifier
# Defining parameter grid (as dictionary)
param_grid_log = {"loss" : ["hinge", "log", "squared_hinge", "modified_huber"], #this actually defines the model you use
              "alpha" : [0.0001, 0.001, 0.01, 0.1],
              "penalty" : ["l2", "l1", "none"]
             }

# Instantiate gridsearch and define the metric to optimize 
gs_log = GridSearchCV(SGDClassifier(random_state=RSEED), param_grid_log, scoring='recall',
                  cv=5, verbose=5, n_jobs=-1)

# Fit gridsearch object to data.. also lets see how long it takes
start = timer()
gs_log.fit(X_train_preprocessed, y_train)
end = timer()
gs_time = end-start

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END alpha=0.0001, loss=hinge, penalty=l2;, score=0.725 total time=   0.0s
[CV 4/5] END alpha=0.0001, loss=hinge, penalty=l2;, score=0.850 total time=   0.0s
[CV 2/5] END alpha=0.0001, loss=hinge, penalty=l2;, score=0.575 total time=   0.0s
[CV 1/5] END alpha=0.0001, loss=hinge, penalty=l1;, score=0.825 total time=   0.0s
[CV 3/5] END alpha=0.0001, loss=hinge, penalty=l2;, score=0.550 total time=   0.0s
[CV 2/5] END alpha=0.0001, loss=hinge, penalty=l1;, score=0.375 total time=   0.0s
[CV 3/5] END alpha=0.0001, loss=hinge, penalty=l1;, score=0.625 total time=   0.0s
[CV 5/5] END alpha=0.0001, loss=hinge, penalty=l1;, score=0.641 total time=   0.0s
[CV 4/5] END alpha=0.0001, loss=hinge, penalty=l1;, score=0.175 total time=   0.0s
[CV 2/5] END alpha=0.0001, loss=hinge, penalty=none;, score=0.550 total time=   0.0s
[CV 5/5] END alpha=0.0001, loss=hinge, penalty=l2;, score=0.564 total time=   0.0s
[CV 3/5] END alpha=0.00

/Users/gunnaroehmichen/neuefische/ds-diabetes-challenge/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/gunnaroehmichen/neuefische/ds-diabetes-challenge/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/gunnaroehmichen/neuefische/ds-diabetes-challenge/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/gunnaroehmichen/neuefische/ds-diabetes-challenge/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWar

In [124]:
# Best score
print('Best score:', round(gs_log.best_score_, 3))

# Best parameters
print('Best parameters:', gs_log.best_params_)

Best score: 0.793
Best parameters: {'alpha': 0.001, 'loss': 'modified_huber', 'penalty': 'l1'}


In [125]:
# we will do this at least twice.. according to DRY we should write a function
def print_pretty_summary(name, model, y_test, y_pred_test):
    print(name)
    print('=======================')
    print('loss: {}'.format(model.loss))
    print('alpha: {}'.format(model.alpha))
    print('penalty: {}'.format(model.penalty))
    accuracy = accuracy_score(y_test, y_pred_test)
    print('Test accuracy: {:2f}'.format(accuracy))
    return accuracy

In [126]:
# Assigning the fitted SGDClassifier model with best parameter combination to a new variable sgd_best
sgd_best = gs_log.best_estimator_

# Making predictions on the test set
y_pred_test = sgd_best.predict(X_test_preprocessed)
# Let us print out the performance of our model on the test set.
sgd_accuracy = print_pretty_summary('SGDClassifier model', sgd_best, y_test, y_pred_test)

SGDClassifier model
loss: modified_huber
alpha: 0.001
penalty: l1
Test accuracy: 0.708333
